In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
import os
from pyspark.streaming import StreamingContext

In [4]:
# Save less frequently and create less subdirectories. I.e. every 120 seconds
ssc = StreamingContext(sc, 120)

C:\Users\u0146617\Downloads\spark\spark\spark-3.5.1-bin-hadoop3\python\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [5]:
lines = ssc.socketTextStream("seppe.net", 7778)

In [6]:
out_dir = f"{os.path.abspath('')}{os.path.sep}saved_stories"
lines.saveAsTextFiles(f"file:///{out_dir}")
print("Saving to", out_dir)

Saving to C:\Users\u0146617\Downloads\spark\spark\notebooks\saved_stories


In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [ ]:
# Don't run this cell unless you want to stop. You should see subdirectories appear in the out_dir
ssc_t.stop()

----- Stopping... this may take a few seconds -----


In [1]:
import pandas as pd
import json
import os
import glob
import sys
import os
import random
random.seed(42)

In [2]:
# Initialize an empty list to store the JSON objects
json_objects = []

# Find all subdirectories under the Data directory
# subdirectories = next(os.walk('Data'))[1]
subdirectories = next(os.walk('data_collection'))[1]

# Loop over all subdirectories
for subdirectory in subdirectories:
    # Construct the full path to the subdirectory
    # subdirectory_path = os.path.join('Data', subdirectory)
    subdirectory_path = os.path.join('data_collection', subdirectory)

    # Find all JSONL files under the current subdirectory
    jsonl_files = glob.glob(os.path.join(subdirectory_path, '*.txt')) + \
                  glob.glob(os.path.join(subdirectory_path, 'part*'))

    # Loop over all JSONL files
    for file in jsonl_files:
        # Open the file and read its contents line by line
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                # Parse the JSON object from the line
                json_obj = json.loads(line)
                # Append the JSON object to the list
                json_objects.append(json_obj)

# Convert the list of JSON objects into a DataFrame
df = pd.DataFrame(json_objects)

df

KeyboardInterrupt: 